In [ ]:
#LEARNING INTRICACIES OF CHATBOTS VIA APPLIED AI

!pip install transformers
#huggingface library

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.9 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertModel

distil_bert = 'distilbert-base-uncased' #name of pretrained model

tokenizer = DistilBertTokenizer.from_pretrained(distil_bert)
model = TFDistilBertModel.from_pretrained(distil_bert)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Extract Features

In [ ]:
#get 768 dense vector representation for any vector given
new = tokenizer.encode("I went to Alliance High School, then the University of Nairobi.") # Convert to a tensorflow tensor
print(new)

input = tf.constant(new) [None, :] # the batch size is 1. If I have multiple sentences, I change the batch size to the number
print(input)
print(type(input))
output = model(input)

print(type(output))
print(len(output))
print(output) # get 768 dimension vector of any sentence of my choice, in this case new
print(model.config)

[101, 1045, 2253, 2000, 4707, 2152, 2082, 1010, 2059, 1996, 2118, 1997, 21124, 1012, 102]
tf.Tensor(
[[  101  1045  2253  2000  4707  2152  2082  1010  2059  1996  2118  1997
  21124  1012   102]], shape=(1, 15), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'transformers.modeling_tf_outputs.TFBaseModelOutput'>
1
TFBaseModelOutput(last_hidden_state=<tf.Tensor: shape=(1, 15, 768), dtype=float32, numpy=
array([[[ 0.27424052, -0.19485079, -0.21896833, ...,  0.02487867,
          0.2973231 ,  0.22755498],
        [ 0.5524583 , -0.26565832, -0.18777221, ..., -0.3294488 ,
          0.58272624, -0.080874  ],
        [ 0.05731173, -0.47184104, -0.09345876, ..., -0.0968073 ,
          0.2752933 , -0.29522496],
        ...,
        [ 0.44652587, -0.17673853, -0.28367707, ..., -0.1605251 ,
         -0.23516345, -0.67236596],
        [ 0.8402448 , -0.09586132, -0.5557103 , ...,  0.3717882 ,
         -0.4271495 , -0.61454064],
        [ 0.8799229 ,  0.04614139, -0.231078

In [ ]:
#Bert has multiple layers within it. The 768 vector received is from the last encoder layer. To get the vectors from the layers in between, I use the following
from transformers import DistilBertConfig

config = DistilBertConfig.from_pretrained(distil_bert, output_hidden_states = True)
new = tokenizer.encode("I went to Alliance High School, then the University of Nairobi.") # Convert to a tensorflow tensor
input = tf.constant(new) [None, :] # the batch size is 1. If I have multiple sentences, I change the batch size to the number
output = model(input)
#concatenates the output into a larger vector
print(model.config)

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.29.2",
  "vocab_size": 30522
}



In [ ]:
from sklearn.linear_model import LogisticRegression


Multi Class Logistic Regression

In [ ]:
X = output[0][:, :, 0:1]
print(X)

tf.Tensor(
[[[ 0.27424052]
  [ 0.5524583 ]
  [ 0.05731173]
  [ 0.01660893]
  [ 0.16738054]
  [ 0.66801953]
  [ 0.32288888]
  [-0.01279473]
  [-0.16823933]
  [-0.09198043]
  [-0.5357172 ]
  [-0.4139384 ]
  [ 0.44652587]
  [ 0.8402448 ]
  [ 0.8799229 ]]], shape=(1, 15, 1), dtype=float32)


In [ ]:
pip install spacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Spacy's Name Entity Recognition
#python -m spacy download en_core_web_sm
import spacy
entityExtractor = spacy.load('en_core_web_sm')

sentence = "Transfer $2000 from my savings account"

entities = entityExtractor(sentence)

for e in entities.ents:
  print(e.text, e.start_char, e.end_char, e.label_)
# import spacy

# entityExtractor = spacy.load('en_core_web_sm')
nlp = entityExtractor

sentence = "Transfer $2000 from my savings account"
sentence1 = "I liked the new Spiderman movie"
sentence2 = "Microsoft and Google are tech juggernauts"
doc = nlp(sentence2)

for token in doc:
    print(token.text, token.ent_iob_, token.ent_type_)



2000 10 14 MONEY
Microsoft B ORG
and O 
Google B ORG
are O 
tech O 
juggernauts O 


In [ ]:
from transformers import BertTokenizer, TFBertForQuestionAnswering
modelName = "bert-large-uncased-whole-word-masking-finetuned-squad"

tokenizer = BertTokenizer.from_pretrained(modelName)

model = TFBertForQuestionAnswering.from_pretrained(modelName)
#BERT finetuned with BertForQuestionAnswering to give answers given questions from a document
print(model)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


Question - Answer

In [ ]:
text = r"""Nairobi, the bustling capital city of Kenya, is a melting pot of cultures that beautifully showcase the diversity of the country. The rich cultural tapestry of Nairobi is a harmonious blend of various ethnic groups, traditions, and influences from across Kenya and beyond. The city is a vibrant hub where people from different backgrounds coexist and celebrate their unique heritage.

Nairobi's culture is characterized by its vibrant music and dance scene. Traditional Kenyan music, such as Benga, blends seamlessly with modern genres like Afro-pop and hip-hop, creating a dynamic and eclectic musical landscape. Dance forms like the popular Odi dance and Maasai traditional dances captivate both locals and visitors alike.

Food is another integral part of Nairobi's culture. The city boasts a wide array of culinary delights, from authentic Kenyan dishes like nyama choma (grilled meat) and ugali (a staple maize meal) to international cuisines brought by the diverse immigrant communities residing in the city.

Art and craftsmanship thrive in Nairobi, with talented artisans showcasing their skills in woodcarving, beadwork, and paintings. The city's art scene has flourished, with numerous galleries and art spaces showcasing both traditional and contemporary African art."""
question1 = r"""What are some of Nairobi's authentic Kenyan dishes?"""

question2 = r"""What characterizes Nairobi's culture"""

question3 = r"""What are the skills of talented artisans in Nairobi?"""

input_text1 = question1 + "[SEP]" + text
input_text2 = question2 + "[SEP]" + text
input_text3 = question3 + "[SEP]" + text
#The BERT module internally adds the [CLS] at the start and [SEP] at the end so that it is [CLS] question [SEP] text [SEP] via hugging face
combined_input_text = input_text1 + "[SEP]" + input_text2 + "[SEP]" + input_text3

# Tokenize the combined input text
input_ids = tokenizer.encode(input_text3)

print(len(input_ids))
print(tokenizer.decode(input_ids))
input = tf.constant(input_ids)[None, :] #1 text, 1 question -> batch size is 1
print(input_ids)

262
[CLS] what are the skills of talented artisans in nairobi? [SEP] nairobi, the bustling capital city of kenya, is a melting pot of cultures that beautifully showcase the diversity of the country. the rich cultural tapestry of nairobi is a harmonious blend of various ethnic groups, traditions, and influences from across kenya and beyond. the city is a vibrant hub where people from different backgrounds coexist and celebrate their unique heritage. nairobi's culture is characterized by its vibrant music and dance scene. traditional kenyan music, such as benga, blends seamlessly with modern genres like afro - pop and hip - hop, creating a dynamic and eclectic musical landscape. dance forms like the popular odi dance and maasai traditional dances captivate both locals and visitors alike. food is another integral part of nairobi's culture. the city boasts a wide array of culinary delights, from authentic kenyan dishes like nyama choma ( grilled meat ) and ugali ( a staple maize meal ) to 

In [ ]:
token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]

print(token_type_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
answer = model(input, token_type_ids = tf.convert_to_tensor([token_type_ids]))

start_logits = answer.start_logits
end_logits = answer.end_logits
print(type(answer))

print(len(answer))

import numpy as np

startScores = np.array(start_logits)
endScores = np.array(end_logits)

print(startScores.shape)

print(endScores.shape)

print(type(startScores))



<class 'transformers.modeling_tf_outputs.TFQuestionAnsweringModelOutput'>
2
(1, 262)
(1, 262)
<class 'numpy.ndarray'>


In [ ]:
input_tokens = tokenizer.convert_ids_to_tokens(input_ids)

print(input_tokens)

['[CLS]', 'what', 'are', 'the', 'skills', 'of', 'talented', 'artisans', 'in', 'nairobi', '?', '[SEP]', 'nairobi', ',', 'the', 'bust', '##ling', 'capital', 'city', 'of', 'kenya', ',', 'is', 'a', 'melting', 'pot', 'of', 'cultures', 'that', 'beautifully', 'showcase', 'the', 'diversity', 'of', 'the', 'country', '.', 'the', 'rich', 'cultural', 'tapestry', 'of', 'nairobi', 'is', 'a', 'harmon', '##ious', 'blend', 'of', 'various', 'ethnic', 'groups', ',', 'traditions', ',', 'and', 'influences', 'from', 'across', 'kenya', 'and', 'beyond', '.', 'the', 'city', 'is', 'a', 'vibrant', 'hub', 'where', 'people', 'from', 'different', 'backgrounds', 'coe', '##xi', '##st', 'and', 'celebrate', 'their', 'unique', 'heritage', '.', 'nairobi', "'", 's', 'culture', 'is', 'characterized', 'by', 'its', 'vibrant', 'music', 'and', 'dance', 'scene', '.', 'traditional', 'kenyan', 'music', ',', 'such', 'as', 'ben', '##ga', ',', 'blend', '##s', 'seam', '##lessly', 'with', 'modern', 'genres', 'like', 'afro', '-', 'pop'

In [ ]:
print(startScores)

[[-5.2334156  -5.761397   -7.648104   -7.470583   -8.35704    -8.524985
  -9.08622    -9.189448   -8.530132   -9.225406   -9.246618   -5.2334037
  -4.813829   -8.240734   -7.2972836  -6.7163615  -8.42058    -6.665034
  -7.369032   -8.196526   -4.8602805  -8.176592   -7.4090686  -7.0875807
  -6.258557   -7.7918925  -8.422655   -6.390717   -8.165416   -6.9060106
  -7.5778966  -8.2544155  -6.8337607  -8.643665   -8.448511   -7.6001115
  -8.155563   -6.6141925  -6.552036   -6.7802806  -7.4921865  -8.554332
  -6.6948657  -8.324768   -7.5968704  -7.090768   -8.581991   -7.2725477
  -8.408125   -6.9292636  -7.159274   -8.073246   -8.637834   -7.087608
  -8.62216    -8.707968   -7.22123    -8.449675   -8.227903   -5.939337
  -8.9455595  -7.922297   -8.079816   -6.8170576  -7.766254   -8.570666
  -7.836867   -6.9848685  -7.951119   -8.568649   -7.2143025  -8.60243
  -8.10554    -8.258264   -8.341936   -8.708241   -8.774267   -8.841835
  -7.369679   -8.429022   -8.058056   -7.572934   -7.785291 

In [ ]:
print(tf.math.argmax(startScores[0], 0))  #tensorflow/math/argmax

tf.Tensor(227, shape=(), dtype=int64)


In [ ]:
print((tf.math.argmax(startScores[0], 0)).numpy())

227


In [ ]:
startIdx = tf.math.argmax(startScores[0], 0).numpy()
endIdx = tf.math.argmax(endScores[0], 0).numpy() + 1
print(startIdx, endIdx)

227 237


In [ ]:
print(" ".join(input_tokens[startIdx:endIdx]))

reconstructed_words = []
for token in input_tokens[startIdx:endIdx]:
    if token.startswith("##"):
        reconstructed_words[-1] += token[2:]
    else:
        reconstructed_words.append(token)

reconstructed_sentence = " ".join(reconstructed_words)
print(reconstructed_sentence)


wood ##car ##ving , bea ##d ##work , and paintings
woodcarving , beadwork , and paintings


In [ ]:
question1 = r"""What are some of Nairobi's authentic Kenyan dishes?"""

question = r"""What characterizes Nairobi's culture"""

question2 = r"""What are the skills of talented artisans in Nairobi?"""

input_text1 = question + "[SEP]" + text
input_text2 = question1 + "[SEP]" + text
input_text3 = question2 + "[SEP]" + text
#The BERT module internally adds the [CLS] at the start and [SEP] at the end so that it is [CLS] question [SEP] text [SEP] via hugging face
input_ids1 = tokenizer.encode(input_text1)
input_ids2 = tokenizer.encode(input_text2)
input_ids3 = tokenizer.encode(input_text3)

input_batch = [input_ids1, input_ids2, input_ids3]

# Pad the batch to the maximum length
max_length = max(len(input_ids1), len(input_ids2), len(input_ids3))
input_batch_padded = [input + [tokenizer.pad_token_id] * (max_length - len(input)) for input in input_batch]

# Find the index of the [SEP] token in the padded input batch
sep_token_id = tokenizer.sep_token_id
sep_token_index = input_batch_padded[0].index(sep_token_id)

# Generate the token_type_ids based on the [SEP] token position
token_type_ids = [0 if i <= sep_token_index else 1 for i in range(max_length)]

print(token_type_ids)

input_batch_tf = tf.constant(input_batch_padded)

answer = model(input_batch_tf, token_type_ids = tf.convert_to_tensor([token_type_ids]))

start_logits = answer.start_logits
end_logits = answer.end_logits

startScores = np.array(start_logits)
endScores = np.array(end_logits)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
input_tokens = [tokenizer.convert_ids_to_tokens(ids) for ids in input_batch_padded]

print(input_tokens)

[['[CLS]', 'what', 'character', '##izes', 'nairobi', "'", 's', 'culture', '[SEP]', 'nairobi', ',', 'the', 'bust', '##ling', 'capital', 'city', 'of', 'kenya', ',', 'is', 'a', 'melting', 'pot', 'of', 'cultures', 'that', 'beautifully', 'showcase', 'the', 'diversity', 'of', 'the', 'country', '.', 'the', 'rich', 'cultural', 'tapestry', 'of', 'nairobi', 'is', 'a', 'harmon', '##ious', 'blend', 'of', 'various', 'ethnic', 'groups', ',', 'traditions', ',', 'and', 'influences', 'from', 'across', 'kenya', 'and', 'beyond', '.', 'the', 'city', 'is', 'a', 'vibrant', 'hub', 'where', 'people', 'from', 'different', 'backgrounds', 'coe', '##xi', '##st', 'and', 'celebrate', 'their', 'unique', 'heritage', '.', 'nairobi', "'", 's', 'culture', 'is', 'characterized', 'by', 'its', 'vibrant', 'music', 'and', 'dance', 'scene', '.', 'traditional', 'kenyan', 'music', ',', 'such', 'as', 'ben', '##ga', ',', 'blend', '##s', 'seam', '##lessly', 'with', 'modern', 'genres', 'like', 'afro', '-', 'pop', 'and', 'hip', '-',

In [ ]:
startIdx = tf.math.argmax(startScores[0], 0).numpy()
endIdx = tf.math.argmax(endScores[0], 0).numpy() + 1

print(startIdx, endIdx)
print(" ".join(input_tokens[startIdx:endIdx]))

88 93

